# Deep Learning Time Series COVID-19 Cases Prediction

## Setup

### **Import libraries** and **packages**

In [65]:
# Auto reload imported module every time a jupyter cell is executed
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [66]:
import numpy as np
import pandas as pd
import scipy
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import requests
import pandas_profiling
from typing import overload
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, GRU, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import RMSprop 
from covid_time_series_prediction.data.data.country_data import country_output

## Data Sourcing

### Data API 

#### By country over time

In [67]:
def fetch_time_series(feature='stringency', start_date='2020-02-14', end_date='2021-02-14'):
    """
    Get stringency time series for each countries requesting API.
    Returns json dict with TS between start_date and end_date like 'YYYY-MM-DD'.
    """
    url = f'https://covidtrackerapi.bsg.ox.ac.uk/api/v2/{feature}/date-range/{start_date}/{end_date}'
    response = requests.get(url)
    if response.status_code != 200:
        return ''
    data = response.json()
    return data

In [68]:
countries_time_series_api = fetch_time_series()

In [69]:
[(k, [c for c in v if c == 'VNM'])  for k, v in countries_time_series_api.items()  if k == 'countries' ]

[('countries', ['VNM'])]

In [70]:
[([([([(vee)  for kaaa, veee  in vee.items() if kaaa in ['date_value', 'confirmed']  ])  for kaa, vee  in ve.items() if kaa =='VNM'   ])  for ka, ve  in v.items() ])  for k, v in countries_time_series_api.items() if k=='data'   ]

[[[[{'date_value': '2020-02-14',
     'country_code': 'VNM',
     'confirmed': 16,
     'deaths': 0,
     'stringency_actual': 45.37,
     'stringency': 45.37,
     'stringency_legacy': 54.76,
     'stringency_legacy_disp': 54.76},
    {'date_value': '2020-02-14',
     'country_code': 'VNM',
     'confirmed': 16,
     'deaths': 0,
     'stringency_actual': 45.37,
     'stringency': 45.37,
     'stringency_legacy': 54.76,
     'stringency_legacy_disp': 54.76}]],
  [[{'date_value': '2020-02-15',
     'country_code': 'VNM',
     'confirmed': 16,
     'deaths': 0,
     'stringency_actual': 47.22,
     'stringency': 47.22,
     'stringency_legacy': 58.33,
     'stringency_legacy_disp': 58.33},
    {'date_value': '2020-02-15',
     'country_code': 'VNM',
     'confirmed': 16,
     'deaths': 0,
     'stringency_actual': 47.22,
     'stringency': 47.22,
     'stringency_legacy': 58.33,
     'stringency_legacy_disp': 58.33}]],
  [[{'date_value': '2020-02-16',
     'country_code': 'VNM',
     'c

In [71]:
[(k, [(ka, [(kaa, vee)  for kaa, vee  in ve.items() if kaa =='USA'   ])  for ka, ve  in v.items() ])  for k, v in countries_time_series_api.items() if k=='data'   ]

[('data',
  [('2020-02-14',
    [('USA',
      {'date_value': '2020-02-14',
       'country_code': 'USA',
       'confirmed': 14,
       'deaths': 0,
       'stringency_actual': 5.56,
       'stringency': 5.56,
       'stringency_legacy': 7.14,
       'stringency_legacy_disp': 7.14})]),
   ('2020-02-15',
    [('USA',
      {'date_value': '2020-02-15',
       'country_code': 'USA',
       'confirmed': 14,
       'deaths': 0,
       'stringency_actual': 5.56,
       'stringency': 5.56,
       'stringency_legacy': 7.14,
       'stringency_legacy_disp': 7.14})]),
   ('2020-02-16',
    [('USA',
      {'date_value': '2020-02-16',
       'country_code': 'USA',
       'confirmed': 14,
       'deaths': 0,
       'stringency_actual': 5.56,
       'stringency': 5.56,
       'stringency_legacy': 7.14,
       'stringency_legacy_disp': 7.14})]),
   ('2020-02-17',
    [('USA',
      {'date_value': '2020-02-17',
       'country_code': 'USA',
       'confirmed': 14,
       'deaths': 0,
       'stringen

#### Country data for a specific day

In [72]:
def fetch_data(country='USA', date='2020-08-14'):
    """
    Get stringency data for one country {ALPHA-3} requesting API.
    Returns json dict with data for country like 'AAA' and specific date and like 'YYYY-MM-DD'.
    """
    url = f'https://covidtrackerapi.bsg.ox.ac.uk/api/v2/stringency/actions/{country}/{date}'
    response = requests.get(url)
    if response.status_code != 200:
        return ''
    data = response.json()
    return data

In [73]:
country_data_api = fetch_data()

In [74]:
[[';'.join([str(kk) for kk, vv in d.items()]) for i, d in enumerate(v) if type(d) == dict and i == 0] for v in country_data_api.values()][0]

['policy_type_code;policy_type_display;policyvalue;policyvalue_actual;flagged;is_general;notes;flag_value_display_field;policy_value_display_field']

In [75]:
[[';'.join([str(vv) for kk, vv in d.items()]) for d in v if type(d) == dict] for v in country_data_api.values()][0]

['C1;School closing;3;3;False;False;According to the New York Times, New York City is the only major school district in the country to open for students part-time this fall. \r\n\r\nSee the article in the New York Times, archived: http://archive.vn/OkUNQ;General;Require closing (all levels)',
 'C2;Workplace closing;2;2;False;False;None;General;Require closing some sectors',
 'C3;Cancel public events;2;2;False;False;None;General;Required',
 'C4;Restrictions on gatherings;4;4;False;False;None;General;Restrictions on gatherings of 10 or fewer people',
 'C5;Close public transport;1;1;False;False;None;General;Recommended',
 'C6;Stay at home requirements;1;1;True;True;None;General;Recommend staying home',
 'C7;Restrictions on internal movement;2;2;False;False;None;General;Required',
 'C8;International travel controls;3;3;None;None;None;Ban',
 'E1;Income support;2;2;False;False;None;All workers;>50% lost income',
 'E2;Debt/contract relief;1;1;None;None;None;Narrow',
 'E3;Fiscal measures;0;0;N

In [76]:
# [';'.join([str(vv) for vv in v]) for v in country_data_api.values()][-1]
[';'.join([str(kk) for kk in v]) for k, v in country_data_api.items()][-1]

'date_value;country_code;confirmed;deaths;stringency_actual;stringency'

### Raw data

#### **Read URL**, **Get CSV files** and **store CSV in local**  *(optional do it at begining or to refresh CSV data)*

##### **get_database_to_csv()** function

In [77]:
def get_database_to_csv(url, csv_list, path='', db_grid=[]) -> list:
    """
    function that take in parameter:
     - a root URL (string) to get the CSV data,
     - a list of CSV files,
     - a path (string) to store CSV in local,
     - a grid (list of list) to add in the CSV filename, URL, local path.     
    and returns the gird updated with the CSVs of the list
    
    """

    ### Create a database grid (list of list) with all CSVs and associated URLs
    # print('db_grid', db_grid)
    #### Data project directory (if empty do not store CSV in local)
    # print('path', path)
    ### Website CSV datasets URL
    # print('url', url)
    #### List of CSVs of Website to retrieve
    # print('csv_list', csv_list)

    #### Length of grid aka number of CSVs already stored in grid
    len_grid = len(db_grid)

    for l in range(len(csv_list)):
        sub_list = []       
        sub_list.append(csv_list[l]) ## 1st pos°: CSV filename
        sub_list.append(url + csv_list[l]) ## 2nd pos°: URL + CSV
        if len(data_dir) > 0: ## store CSV in local
            sub_list.append(data_dir + csv_list[l]) ## 3rd pos°: local data path + CSV
            !curl -L "{url + csv_list[l]}" > {data_dir + csv_list[l]} ## curl <URL>/<CSV> => <path>
        # print('sub_list', sub_list)
        db_grid.append(sub_list)

    ### Return a database grid (list of list) with all CSVs and associated URLs
    return db_grid

#### **Get database to csv** with **get_database_to_csv()** function

In [78]:
#### Data project directory
data_dir = '../data/raw_data/'

### Oxford Master data time series URL
url_root_oxford = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/'

#### List of CSVs of Oxford database Feel free to add more feature...
## index_strigency missed!
## 'E3;Fiscal measures;' missed!
## 'E4;International support;' missed!
## 'H3;Contact tracing;' missed!
## 'H4;Emergency investment in healthcare!
## 'H5;Investment in vaccines missed!
## 'V1;Vaccine Prioritisation missed!
## 'V2;Vaccine Availability missed!
## 'V3;Vaccine Financial Support!
## 'V4;Mandatory Vaccination missed!
csv_list = ['confirmed_cases.csv', 'confirmed_deaths.csv',
            'government_response_index_avg.csv', 'stringency_index_avg.csv', 
            'containment_health_index_avg.csv', 'economic_support_index.csv',
            'c1m_school_closing.csv', 'c2m_workplace_closing.csv',
            'c3m_cancel_public_events.csv', 'c4m_restrictions_on_gatherings.csv', 
            'c5m_close_public_transport.csv', 'c6m_stay_at_home_requirements.csv',
            'c7m_movementrestrictions.csv', 'c8ev_internationaltravel.csv',
            'e1_income_support.csv', 'e2_debtrelief.csv',
            'h1_public_information_campaigns.csv', 'h2_testing_policy.csv',
            'h3_contact_tracing.csv', 'h6m_facial_coverings.csv',
            'h7_vaccination_policy.csv', 'h8m_protection_of_elderly_ppl.csv'
           ] ## ; print('csv_list', csv_list, 'len(csv_list)', len(csv_list))
    
### Vacinations Dataset URLs
url_root_vaccinations = 'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/'

#### List of CSVs of Vaccinations database
csv_list_vax = ['vaccinations.csv', 'vaccinations-by-age-group.csv'] ## ; print('csv_list', csv_list_vax, 'len(csv_list)', len(csv_list_vax))

### Create a database grid all CSVs and associated URLs from Oxford website
db_grid = get_database_to_csv(url_root_oxford, csv_list, data_dir)
### Insert into database grid all CSVs and associated URLs from vaccinations website
db_grid = get_database_to_csv(url_root_vaccinations, csv_list_vax, data_dir, db_grid)
# print('db_grid', db_grid)

# Stack all csl in the list
csv_list += csv_list_vax ## ; print('csv_list', csv_list)

# transform list into dict:
csv = dict(zip(csv_list, [v[0] for v in enumerate(csv_list)])) ## ; print ('csv', csv) ## if v[1] == 'containment_health_index_avg.csv' 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1999k  100 1999k    0     0  3433k      0 --:--:-- --:--:-- --:--:-- 3465k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1587k  100 1587k    0     0  2801k      0 --:--:-- --:--:-- --:--:-- 2820k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1924k  100 1924k    0     0  3575k      0 --:--:-- --:--:-- --:--:-- 3603k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1887k  100 1887k    0     0  3587k      0 --:--:-- --:--:-- --:--:-- 3615k
  % Total    % Received % Xferd  Average Speed   Tim

### **Read CSV** and **Set raw dataframe**

### **Read raw CSV** and **Set dataframe**

In [79]:
df_raw_gov_response = pd.read_csv(data_dir + 'government_response_index_avg.csv')
df_raw_health = pd.read_csv(data_dir + 'containment_health_index_avg.csv')
df_raw_economic = pd.read_csv(data_dir + 'economic_support_index.csv')

#### Vaccination
df_raw_vaccination = pd.read_csv(data_dir + 'vaccinations.csv')
df_raw_ages = pd.read_csv(data_dir + 'vaccinations-by-age-group.csv')


#### Data Frame target
df_raw_cases = pd.read_csv(data_dir + 'confirmed_cases.csv')
df_raw_deaths = pd.read_csv(data_dir + 'confirmed_deaths.csv')

#### Data multiple
df_raw_school_closing=pd.read_csv(data_dir + 'c1m_school_closing.csv')
df_raw_workplace_closing=pd.read_csv(data_dir + 'c2m_workplace_closing.csv')
df_raw_cancel_public_event=pd.read_csv(data_dir + 'c3m_cancel_public_events.csv')
df_raw_restriction_on_gathering=pd.read_csv(data_dir + 'c4m_restrictions_on_gatherings.csv')
df_raw_stay_at_home=pd.read_csv(data_dir + 'c6m_stay_at_home_requirements.csv')
df_raw_international_travel=pd.read_csv(data_dir + 'c6m_stay_at_home_requirements.csv')
df_raw_goverment_response=pd.read_csv(data_dir + 'government_response_index_avg.csv')
df_raw_facial_covering=pd.read_csv(data_dir + 'h6m_facial_coverings.csv')
df_raw_vacination_policy=pd.read_csv(data_dir + 'h7_vaccination_policy.csv')


### **Read out CSV** and **Set dataframe**

In [80]:
#### CSV Data out project directory
csv_dir = '../data/csv_out/'
df_usa_index =  pd.read_csv(csv_dir + 'usa_index.csv')
df_usa_indicator =  pd.read_csv(csv_dir + 'usa_indicator.csv')
# df_usa_index, df_usa_indicator

## Set cleaned DataFrames

In [81]:
df_ts_usa_index =  df_usa_index.copy()
df_ts_usa_indicator =  df_usa_indicator.copy()

## Exploratory Data Analyzis

### Time Series **Analysis** & **Preparation** to training

In [82]:
# Set target feature to total deaths
y_seq = 'total_deaths'
df_ts_usa_index = df_usa_index.copy()
df_ts_usa_index.drop(columns=['date', 'new_deaths', 'new_cases'],inplace=True)

df_ts_usa_index = df_ts_usa_index.fillna(0)
print(df_ts_usa_index.shape[1])
df_ts_usa_index

9


,containment_and_health,stringency,economics_sup,total_cases,total_deaths,gov_response,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred
0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0
1,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0
2,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0
3,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0
4,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0
...,...,...,...,...,...,...,...,...,...
977,0.0,0.0,0.0,188464761.0,2089425.0,2146.68,79.24,67.58,39.5
978,0.0,0.0,0.0,188503831.0,2089567.0,2146.68,79.25,67.58,39.5
979,0.0,0.0,0.0,188761844.0,2090809.0,2146.68,79.25,67.58,39.5
980,0.0,0.0,0.0,103838653.0,0.0,2146.68,0.00,0.00,0.0


### Time Series Analysis *(optional)*

In [83]:
ts_cases = df_raw_cases.drop(columns=['country_name','region_code','region_name','jurisdiction','Unnamed: 0'])
ts_cases = ts_cases.groupby('country_code').agg('sum')
ts_cases.transpose()
ts_cases.columns.name = 'Dates'
ts_cases = ts_cases.fillna(0)
# ts_cases.index = pd.to_datetime(ts_cases.index)

In [84]:
ts_cases = ts_cases.transpose()
ts_cases.head()

country_code,ABW,AFG,AGO,ALB,AND,ARE,ARG,AUS,AUT,AZE,...,USA,UZB,VEN,VIR,VNM,VUT,YEM,ZAF,ZMB,ZWE
Dates,,,,,,,,,,,,,,,,,,,,,
01Jan2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
02Jan2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
03Jan2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
04Jan2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
05Jan2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [85]:
ts_cases.shape

(983, 187)

In [86]:
vn_ts_cases

NameError: name 'vn_ts_cases' is not defined

In [ ]:
vn_ts_cases = vn_ts_cases.transpose()

In [ ]:
df_raw_cases.isna()

In [ ]:
# get VietNam country dataset
vn_data = df_raw_cases.loc[df_raw_cases['country_code'] == 'VNM'].copy()

vn_data.head()

In [ ]:
# %%time
# vn_data.profile_report()

# RNN MODEL

## Recurrent Neural Network (sequences data) modeling

### Samples/Sequences, Observations, Features

X.shape = (n_SEQUENCES, n_OBSERVATIONS, n_FEATURES)

y = RNN(X)

❗️ Notation $X_{i,j}^{t}$

 $_{i}$ is the sample/sequence
 
 $_{j}$ is the feature measured
 
 $^{t}$ is the time at which the observation is seen

In [ ]:
n_seq = 50 # ts_cases.shape[0] - 1 # nb of countries (samples)
n_obs = 15 # 15 days of training periiod (observations)
n_feat = df_ts_usa_index.shape[1] # 1 # 1feature: covid cases
n_pred = 1 # nb of days of prediction
n_feat

In [ ]:
# 3. Training
from tensorflow.keras.callbacks import History
def train_rnn_model(model, patience=5, epochs=200) -> History:
    es = EarlyStopping(monitor = 'val_loss',
                    patience = patience,
                    verbose = 0,
                    restore_best_weights = True)
    # The fit
    history =  model.fit(X_train,
            y_train, 
            validation_split=0.1, # Auto split for validation data
                ## validation_data = (X_val, y_val), # To be created manually if needed
            batch_size = 16,
            epochs = epochs,
            callbacks = [es],
            verbose=1)
    print(type(history))
    return history


# print(type(overfit_es), overfit_es)
# if overfit_es:
#     print("early stopping")
#     history = train_rnn_model(patience=overfit_es)
# else:
# print("No early stopping")
# history = train_rnn_model()

In [ ]:
n_seq = 400 # ts_cases.shape[0] - 1 # nb of countries (samples)
n_obs = 150 # 15 days of training periiod (observations)
n_feat = df_ts_usa_index.shape[1] # 1 # 1feature: covid cases
n_pred = 1 # nb of days of prediction
# length of a sequence
len_seq = n_obs + n_pred
# Ex: 16 = 15 + 1
len_seq, n_feat

X = np.array(ts).astype(np.float32)

y = np.array([y_a, y_b, y_c]).astype(np.float32)



### Prerequisites:

- **retrieve dataset** from Alberto

    - **clean dataset**: 
        
        - **drop first lines == 0** *(before Covid arrived)*
        
        - **check Nan**: 
- **strategy 1 country by country** sequences split as follow:

- **strategy 2 one sequence per country**:
    - **split X train, set** 
    - **Pad sequences**
    - **create one csv per country**

## Training strategies:
- Get NB dataset (cleaned) from Alberto & Sumedha
- 1/ Indicator in precentage %
- 2/ Indicator as categorical labels
- Run same RNN model in parallel with Kim & Thomas
- Identify best dataset
- Parameters to fit:
    - increase **nb of sequences**
    - train series modulation (ex: [50, 150, 200, 300, 400 nb of days = n_obs]) < take time to compute
    - **learning_rate** in Optimizer(parameters)
    - model layers architecture (**simple** -> complex) (less data -> more data) (print(loss) function check lecture)
        > LSTM
        > Dense
       (> LSTM
        > LSTM
        > Dense)
     >> **try to overfit** the model with the loss (train over val) or (early_stopping)
     >> **(X_val, y_val)**

#### Create sequences (`X`,`y`)

##### **subsample_sequence(df, length)**

function that given a full dataframe `df`:
- Create a sub sequences df, with long length?

In [ ]:
def subsample_sequence(df, length) -> pd.DataFrame:
    """
    Given the initial dataframe `df`, return a shorter dataframe sequence of length `length` (eg n_obs).
    This shorter sequence should be selected at random
    """
    last_possible = df.shape[0] - length
    # How to split sequences? we could do it manually...
    random_start = np.random.randint(0, last_possible)
    df_sample = df[random_start: random_start+length]
    
    return df_sample

In [ ]:
# Test it 
# assert subsample_sequence(vn_ts_cases, 10).shape  == (10, 1)
# assert subsample_sequence(vn_ts_cases, 400).shape == (400, 1)
subsample_sequence(df_ts_usa_index, 10).shape, subsample_sequence(df_ts_usa_index, 400).shape

#### Pad `X` missing values with mean values

##### **split_subsample_sequence(df,  length, sequence='VNM', df_mean=None)**

function that given a full dataframe `df`:
- Create a sub sequences df
- Stores the value of the covid deaths* (or cases) of the last day as your variable array `y`
- Stores all features of previous days as a variable `X`
- Returns (`X`, `y`)

* *'VNM_covd_deaths'* preference for death prediction, then switch on cases *'VNM_covid_cases'*

In [ ]:
def split_subsample_sequence(df,  length, sequence='VNM', y_size=1) -> tuple:
    '''
    Create one single random (X_sample, y_sample)
    containing one sequence each of length `length`
    ToDo: Adapt the y size=-1'''
    # Trick to save time during potential recursive calls
    # if df_mean is None:
    #     df_mean = df.mean()
    df_subsample = subsample_sequence(df, length)
    y_sample = df_subsample.iloc[length - y_size][sequence] # ['VNM'] ['VNM_covid_cases', 'VNM_covd_deaths'] 
    # Case y_sample is NaN: redraw !
    # if y_sample != y_sample: # A value is not equal to itself only for NaN
    #         X_sample, y_sample = split_subsample_sequence(df, length, df_mean) # Recursive call !!!
    #         return np.array(X_sample), np.array(y_sample)    
    X_sample = df_subsample[0:length - y_size]
    # Case X_sample has some NaNs
    # if X_sample.isna().sum().sum() !=0:
    #    X_sample = X_sample.fillna(compute_means(X_sample, df_mean))
    #    X_sample = X_sample.values

    return np.array(X_sample), np.array(y_sample)

In [ ]:
# Test it
print(y_seq)
(X_sample, y_sample) = split_subsample_sequence(df_ts_usa_index, sequence=y_seq, length=len_seq, y_size=n_pred)
X_sample.shape, y_sample.shape

#### Generates an entire dataset of multiple subsamples with shape $(X, y)$

##### **get_X_y(df, n_sequences, length)**

function to generates an entire dataset of multiple subsamples suitable for RNN, that is, $(X, y)$ of shape:

```python
X.shape = (n_sequences, length, n_features)
y.shape = (n_sequences, )
```

In [ ]:
def get_X_y(df, n_sequences, length, sequence='VNM') -> tuple:
    '''Return a list of samples (X, y)'''
    X, y = [], []

    for i in range(n_sequences):
        (xi, yi) = split_subsample_sequence(df, length, sequence=sequence)
        X.append(xi)
        y.append(yi)
        
    X = np.array(X)
    y = np.array(y)

    return X, y

Generate your dataset $(X, y)$ of `50` sequences, each of `15` observations + the covid cases at the 16-th day to predict

n_seq = 50 # ts_cases.shape[0] - 1 # nb of countries (samples)
n_obs = 15 # 15 days of training periiod (observations)
n_feat = 1 # 1feature: covid cases
n_pred = 1 # nb of days of prediction
len_seq = 16 # length of a sequence (len_seq = n_obs + n_pred/ Ex: 16 = 15 + 1)

In [ ]:
X, y = get_X_y(df_ts_usa_index, n_sequences=n_seq, length=len_seq, sequence=y_seq)
print(X.shape) ; print(y.shape)

##### Test it
(X_sample, y_sample) = split_subsample_sequence(vn_ts_cases, length=len_seq, y_size=n_pred)
###### assert X_sample.shape == (n_obs,n_feat)
###### assert y_sample.shape == ()
X_sample.shape, y_sample.shape

# Check your code below
assert X.shape == (50, 15, 1)
assert y.shape == (50, )
assert np.isnan(X).sum() == 0

### How to split sequences?

- randomly or

- manually

### Samples/Sequences, Observations, Features

In [ ]:
# 3. Training
def train_rnn_model_val_set(model, patience=20, epochs=200):
    es = EarlyStopping(monitor = 'val_loss',
                    patience = patience,
                    verbose = 0,
                    restore_best_weights = True)
    # The fit
    history =  model.fit(X_train,
            y_train, 
            ## validation_split=0.1, # Auto split for validation data
            validation_data = (X_val, y_val), # To be created manually if needed
            batch_size = 16,
            epochs = epochs,
            callbacks = [es],
            verbose=1)
    return history


print(type(overfit_es), overfit_es)
# if overfit_es:
#     print("early stopping")
#     history = train_rnn_model(patience=overfit_es)
# else:
# print("No early stopping")
# history = train_rnn_model()

### Split train

In [ ]:
# -1. Train splitting
# Let's keep the last 20% of the values out for testing purposes
train_size = 0.8 ## 80% of dataset to train
index = round(train_size * df_ts_usa_index.shape[0])

df_train = df_ts_usa_index.iloc[:index]
df_test = df_ts_usa_index.iloc[index:]
df_train.shape, df_test.shape

Let's not cross-validate in this challenge to start with 🤯 
- Separate `df` into `df_train` and `df_test` such that the first 80% of the dataframe is in the training, and the last 20% in the test set.
- Then generate (`X_train`, `y_train`) from `df_train` and (`X_test`, `y_test`) from `df_test`
- Ensure that `X_train.shape == (50, 15, 1)`

len_ = int(0.8*ts_cases.shape[0])
df_train = ts_cases[:len_] ; df_test = ts_cases[len_:]
df_train.shape, df_test.shape

In [ ]:
n_seq_test = n_seq // 3 ; n_seq_test

In [ ]:
X_train, y_train = get_X_y(df_train, n_seq, len_seq, sequence=y_seq)
X_test, y_test = get_X_y(df_test, n_seq_test, len_seq, sequence=y_seq)

In [ ]:
X_train.shape, X_test.shape

### Normalization layer

In [ ]:
# 0. The Normalization Layer
normalizer = Normalization()  # Instantiate a "normalizer" layer
normalizer.adapt(X_train) # "Fit" it on the train set

### RNN model #1 architecture

In [ ]:
# 1. The Architecture
rnn_model = Sequential()
rnn_model.add(normalizer) # Using the Normalization layer to standardize the datapoints during the forward pass
rnn_model.add(SimpleRNN(units=20, activation='tanh', return_sequences=True))  ## , input_shape=(?,?))) without a Normalizer layer
rnn_model.add(SimpleRNN(units=20, activation='tanh'))  ## 
rnn_model.add(Dense(10, activation = 'relu')) ## add 1 or more 'relu' layers
# model.add(layers.Dropout(0.3)) ## if RNN model over-fit
rnn_model.add(Dense(n_pred, activation = 'linear'))

### Compile model #1 with 'rmsprop'

In [ ]:
# 2. Compiling with 'rmsprop' rather than 'adam' (recommended)
optimizer = RMSprop(
                learning_rate=0.001,
                rho=0.9,
                momentum=0.0,
                epsilon=1e-07,
                centered=False
            )
rnn_model.compile(loss='mse',
              optimizer= optimizer, # optimizer='rmsprop'    <- adapt learning rate
                 metrics='mape')  # Recommended optimizer for RNNs

In [ ]:
rnn_model.summary()

### Train model #1

In [ ]:
history = train_rnn_model(rnn_model, patience=5, epochs=200)
plt.plot(history.history['mape'])
plt.plot(history.history['val_mape'])
plt.show();
type(history)

In [ ]:
train_series = [50, 150, 200, 300, 400]
overfit_es =   [50, 66, 26, 45, 60 ]
print('type(overfit_es), overfit_es', type(overfit_es), overfit_es)
# if overfit_es:
#     print("early stopping")
#     history = train_rnn_model(patience=overfit_es)
# else:
# print("No early stopping")
for i in range(len(train_series)):
    history = train_rnn_model(model=rnn_model, epochs=train_series[i], patience=overfit_es[i])
    plt.plot(history.history['mape'])
    plt.plot(history.history['val_mape'])
    plt.show();

### RNN model #2 architecture

#### 🚀 The **LSTM (= Long Short Term Memory)** with their ability to *avoid the vanishing gradient problem*, should be preferred over a SimpleRNN.

In [ ]:
# 1. The Architecture
"""   - 2nd model layers architecture (simple -> complex) (less data -> more data) (print(loss) function check lecture)
> LSTM
"""
rnn_model_2 = Sequential()
rnn_model_2.add(normalizer) # Using the Normalization layer to standardize the datapoints during the forward pass
# Input len(train) (input_shape=(?,?))
rnn_model_2.add(SimpleRNN(units=30, activation='tanh', return_sequences=True))  ## , input_shape=(?,?))) without a Normalizer layer
# output return sequences = True
rnn_model_2.add(SimpleRNN(units=20, activation='tanh', return_sequences=True))  ## LSTM
rnn_model_2.add(SimpleRNN(units=10, activation='tanh'))  ## LSTM
rnn_model_2.add(Dense(10, activation = 'relu')) ## add 1 or more 'relu' layers
# Output 10 only, no more RNN just dropout()
# rnn_model_2.add(layers.Dropout(0.3)) ## if RNN model over-fit
rnn_model_2.add(Dense(n_pred, activation = 'linear'))

### Compile model #2 with 'rmsprop'

In [ ]:
# 2. Compiling with 'rmsprop' rather than 'adam' (recommended)
optimizer = RMSprop(
                learning_rate=0.001,
                rho=0.9,
                momentum=0.0,
                epsilon=1e-07,
                centered=False
            )
rnn_model_2.compile(loss='mse',
              optimizer= optimizer, # optimizer='rmsprop'    <- adapt learning rate
                 metrics='mape')  # Recommended optimizer for RNNs
rnn_model_2.summary()

### Train model #2

In [ ]:
trai### Train model #1n_series = [50, 150, 200, 300, 400]
overfit_es =   [25, 60, 26, 45, 60 ]
print('type(overfit_es), overfit_es', type(overfit_es), overfit_es)
# if overfit_es:
#     print("early stopping")
#     history = train_rnn_model(patience=overfit_es)
# else:
# print("No early stopping")
for i in range(len(train_series)):
    history = train_rnn_model(model=rnn_model_2, epochs=train_series[i], patience=overfit_es[i])
    plt.plot(history.history['mape'])
    plt.plot(history.history['val_mape'])
    plt.show();

### RNN model #3 architecture 

In [ ]:
# 1. The Architecture
"""   - 3rd model layers architecture (simple -> complex) (less data -> more data) (print(loss) function check lecture)
> LSTM
"""
rnn_model_3 = Sequential()
rnn_model_3.add(normalizer) # Using the Normalization layer to standardize the datapoints during the forward pass
# Input len(train) (input_shape=(?,?))
rnn_model_3.add(LSTM(units=30, activation='tanh'))  ## , input_shape=(?,?))) without a Normalizer layer
# output return sequences = True
rnn_model_3.add(Dense(10, activation = 'relu')) ## add 1 or more 'relu' layers
# Output 10 only, no more RNN just dropout()
# rnn_model_3.add(layers.Dropout(0.3)) ## if RNN model over-fit
rnn_model_3.add(Dense(n_pred, activation = 'linear'))

### Compile model #3 with 'rmsprop'

In [ ]:
# 2. Compiling with 'rmsprop' rather than 'adam' (recommended)
optimizer = RMSprop(
                learning_rate=0.001,
                rho=0.9,
                momentum=0.0,
                epsilon=1e-07,
                centered=False
            )
rnn_model_3.compile(loss='mse',
              optimizer= optimizer, # optimizer='rmsprop'    <- adapt learning rate
                 metrics='mape')  # Recommended optimizer for RNNs
rnn_model_3.summary()

### Train model #2

In [ ]:
# 3. Training
from typing import overload


def train_rnn_model(model, patience=20, epochs=200):
    es = EarlyStopping(monitor = 'val_loss',
                    patience = patience,
                    verbose = 0,
                    restore_best_weights = True)
    # The fit
    history =  model.fit(X_train,
            y_train, 
            validation_split=0.1, # Auto split for validation data
                ## validation_data = (X_val, y_val), # To be created manually if needed
            batch_size = 16,
            epochs = epochs,
            callbacks = [es],
            verbose=1)
    return history


print(type(overfit_es), overfit_es)
# if overfit_es:
#     print("early stopping")
#     history = train_rnn_model(patience=overfit_es)
# else:
# print("No early stopping")
# history = train_rnn_model()

In [ ]:
# min(history.history['mape'])

# print("adjust early stopping")
# overfit_es = [d[0]+1 for d in enumerate(history.history['mape']) if d[1] == min(history.history['mape'])][0]
# overfit_es

In [ ]:
min(history.history['mape']), max(history.history['mape']), history.history['mape'] # blue line

In [ ]:
max(history.history['val_mape']), history.history['val_mape'] # orange line

### Evaluate model #1

In [ ]:
# 4. Evaluating
# The prediction (one per sequence/city)
y_pred = rnn_model.predict(X_test) 

In [ ]:
y_pred.shape

## Time Series Forecasting

In [ ]:
# Check your code below
assert y_pred.shape == (n_seq_test, n_pred)